# SETTINGS

In [1]:
FOLDER = "Radiate_Image_Dataset/tiny_foggy/"

## Create lidar_to_camera_frame.txt

In [2]:
def create_frames(lines):
    frames=[]
    for line in lines:
        temp = line.split()
        if len(temp[3])<20:
            dif = 20-len(temp[3])
            zeros = '0'*dif
            num = temp[3].split(".")
            num[1] = zeros + num[1]
            new_num = num[0]+"."+num[1]
            temp[3] = new_num   
        frames.append([temp[1],temp[3]])
    return frames

def add_zeros(frame):
    frame=str(frame)
    add_zero = 6- len(frame)
    for i in range(add_zero):
        frame = "0"+frame
    return frame
    
    
def correct_file(file,frames):
    for frame in frames:
        file.write("Frame: "+frame[0]+" Time: "+frame[1]+"\n")
    return
    
camera_txt = open(FOLDER + "zed_right.txt","r")
lines = camera_txt.readlines()
camera_frames=create_frames(lines)
camera_txt.close()

camera_txt = open(FOLDER + "zed_right.txt","w")
correct_file(camera_txt,camera_frames)
camera_txt.close()

lidar_txt = open(FOLDER + "velo_lidar.txt","r")
lines = lidar_txt.readlines()
lidar_txt.close()
lidar_frames = create_frames(lines)

lidar_txt = open(FOLDER + "velo_lidar.txt","w")
correct_file(lidar_txt,lidar_frames)
lidar_txt.close()

radar_txt = open(FOLDER + "Navtech_Cartesian.txt","r")
lines = radar_txt.readlines()
radar_txt.close()
radar_frames=create_frames(lines)

radar_txt = open(FOLDER + "Navtech_Cartesian.txt","w")
correct_file(radar_txt,radar_frames)
radar_txt.close()


camera_id=0
lidar_camera=[]
for lidar_id in range(len(lidar_frames)):
    if camera_id>=len(camera_frames):
        break
    c_time = camera_frames[camera_id][1]
    l_time = lidar_frames[lidar_id][1]
    
    if float(l_time)<float(c_time)-0.1:
        continue

    while(True):
        if float(l_time)>float(c_time)+0.1:
            camera_id+=1
            c_time = camera_frames[camera_id][1]
            continue
        break
    
    if float(l_time)<float(c_time)-0.1:
        continue
    lidar_camera.append([lidar_frames[lidar_id][0],camera_frames[camera_id][0],c_time])
    camera_id+=1

print(len(lidar_camera))

radar_id=0
lidar_camera_radar=[]
for lidar_cam_id in range(len(lidar_camera)):
    if radar_id>=len(radar_frames):
        break
    cl_time = lidar_camera[lidar_cam_id][2]
    r_time = radar_frames[radar_id][1]
    if float(cl_time)<float(r_time)-0.1:
        continue

    while(True):
        if float(cl_time)>float(r_time)+0.1:
            radar_id+=1
            r_time = radar_frames[radar_id][1]
            continue
        break

    if float(cl_time)<float(r_time)-0.1:
        continue
    lidar_camera_radar.append([lidar_camera[lidar_cam_id][0],lidar_camera[lidar_cam_id][1],r_time])
    radar_id+=1

print(len(lidar_camera_radar))
file = open(FOLDER+"lidar_to_camera_frame.txt","w")
for frames in lidar_camera_radar:
    lidar_frame = add_zeros(frames[0])
    camera_frame = add_zeros(frames[1])
    timestamp = str(frames[2])
    file.write("Lidar_Frame: "+lidar_frame+" Camera_Frame: "+camera_frame+" Timestamp: "+timestamp+"\n")
    
file.close()

35
14


## Export coco annoation

In [5]:
import radiate_sdk.radiate as rd
import numpy as np
import os 
from tqdm import tqdm

def assign_class_num(class_name):
    if class_name == "car":
        return 0
    elif class_name == "van":
        return 1
    elif class_name == "truck":
        return 2
    elif class_name == "bus":
        return 3
    elif class_name == "motorbike":
        return 4
    elif class_name == "pedestrian":
        return 5
    elif class_name == "group_of_pedestrians":
        return 6
    elif class_name == "bicycle":
        return 7
    else:
        print(class_name)


seq = rd.Sequence(FOLDER)

frames_txt = open(FOLDER+"lidar_to_camera_frame.txt","r")
frames = frames_txt.readlines()

try:
    os.mkdir(FOLDER+'coco_annotations')
except:
    print("coco_annotations file already exists")

only_ann_file = open(FOLDER+"coco_annotations/only_with_annotations.txt","w")

for line in tqdm(frames):
    temp = line.split()
    lidar,camera,timestamp=temp[1],temp[3],temp[5]

    file = open(FOLDER+"coco_annotations/"+lidar+"+"+camera+".txt","w")
    
    info = seq.get_from_timestamp(float(timestamp))

    
    if info == {}:
        continue
    
    bboxes = info['annotations']['camera_right_rect']

        
    for i in range(len(bboxes)):
        bbox = [bboxes[i]]
        
        bbox_array = bbox[0]['bbox_3d']
        if bbox_array.size==0:
            continue
        xmin = np.min(bbox_array[:,0])
        ymin = np.min(bbox_array[:,1])
        xmax = np.max(bbox_array[:,0])
        ymax = np.max(bbox_array[:,1])
        
        class_num = assign_class_num(bbox[0]['class_name'])

        width = abs(xmax-xmin)
        height = abs(ymax-ymin)
        
        coco_bbox=[class_num, xmin,ymin,width,height]

        coco_bbox_str = str(coco_bbox[0])+" "+str(coco_bbox[1])+" "+str(coco_bbox[2])+" "+str(coco_bbox[3])+" "+str(coco_bbox[4])+"\n"
        file.write(coco_bbox_str)


    file.close()
    if len(bboxes)>0:
        only_ann_file.write(lidar+"+"+camera+".png\n")
only_ann_file.close()

print("coco_annotation file exported")

100%|██████████| 14/14 [00:01<00:00, 11.84it/s]

coco_annotation file exported


## Create img folder

In [ ]:
import os

try:
    os.mkdir(FOLDER+'fused_images')
    os.mkdir(FOLDER+'fused_images/RGB')
    os.mkdir(FOLDER+'fused_images/RGB/img')
    os.mkdir(FOLDER+'fused_images/RGB_distance')
    os.mkdir(FOLDER+'fused_images/RGB_distance/img')
    
except:
    print("fused_images file already exists")



## Create fused images

In [7]:
import cv2
import radiate_sdk.radiate as rd
import numpy as np

def export_image(seq,lidar,camera,mode):

        data = np.genfromtxt(FOLDER+"velo_lidar/"+lidar+".csv",delimiter=",")

        image = cv2.imread(FOLDER+"zed_right/"+camera+".png")
        image_array = np.array(image)

        if (mode=="none"):
            cv2.imwrite(FOLDER+"fused_images/RGB/img/"+lidar+"+"+camera+".png",image)
        

        elif (mode == "projection_same"):
            lidar_proj = seq.project_lidar(data,seq.calib.LidarToRight,seq.calib.right_cam_mat,"same")
            fusion = seq.overlay_camera_lidar(image_array,lidar_proj)
            cv2.imwrite(FOLDER+"fused_images/RGB_projection_same/img/"+lidar+"+"+camera+".png",fusion)

        elif (mode == "projection_pseudo"):
            lidar_proj = seq.project_lidar(data,seq.calib.LidarToRight,seq.calib.right_cam_mat,"pseudo_distance")
            fusion = seq.overlay_camera_lidar(image_array,lidar_proj)
            cv2.imwrite(FOLDER+"fused_images/RGB_projection_pseudo/img/"+lidar+"+"+camera+".png",fusion)
            

        elif (mode == "pseudo"):
            lidar_proj = seq.project_lidar(data,seq.calib.LidarToRight,seq.calib.right_cam_mat,"pseudo_distance")
            dist = np.mean(lidar_proj,axis=2)
            dist = np.expand_dims(dist,axis=2)
            fusion = np.dstack((image_array,dist))
            cv2.imwrite(FOLDER+"fused_images/RGB_pseudo/img/"+lidar+"+"+camera+".png",fusion)
            
        elif (mode == "same"):
            lidar_proj = seq.project_lidar(data,seq.calib.LidarToRight,seq.calib.right_cam_mat,"same")
            dist = np.mean(lidar_proj,axis=2)
            dist = np.expand_dims(dist,axis=2)
            fusion = np.dstack((image_array,dist))
            cv2.imwrite(FOLDER+"fused_images/RGB_same/img/"+lidar+"+"+camera+".png",fusion)

        elif (mode == "distance"):
            lidar_proj = seq.project_lidar(data,seq.calib.LidarToRight,seq.calib.right_cam_mat,"distance")
            dist = np.expand_dims(lidar_proj,axis=2)
            fusion = np.dstack((image_array,dist))
            cv2.imwrite(FOLDER+"fused_images/RGB_distance/img/"+lidar+"+"+camera+".png",fusion)

        return

In [8]:
from tqdm import tqdm

seq = rd.Sequence(FOLDER)

frames_txt = open(FOLDER+"lidar_to_camera_frame.txt","r")
frames = frames_txt.readlines()

for line in tqdm(frames):
    temp = line.split()
    lidar,camera,timestamp=temp[1],temp[3],temp[5]

    export_image(seq,lidar,camera,"none")
    #export_image(seq,lidar,camera,"projection_same")
    #export_image(seq,lidar,camera,"same")
    export_image(seq,lidar,camera,"distance")

100%|██████████| 14/14 [00:01<00:00,  9.80it/s]


## Delete unnecessary files

In [9]:
import shutil
import os

try:
    shutil.rmtree(FOLDER+'GPS_IMU_Twist')
    os.remove(FOLDER+'GPS_IMU_Twist.txt')
except:
    print("GPS has already been removed")

try:
    
    shutil.rmtree(FOLDER+'Navtech_Cartesian')
    shutil.rmtree(FOLDER+'Navtech_Polar')
    shutil.rmtree(FOLDER+'zed_left')
    shutil.rmtree(FOLDER+'zed_right')
    shutil.rmtree(FOLDER+'annotations')
    shutil.rmtree(FOLDER+'velo_lidar') 

    
    os.remove(FOLDER+'Navtech_Cartesian.txt')
    os.remove(FOLDER+'Navtech_Polar.txt')
    os.remove(FOLDER+'zed_left.txt')
    os.remove(FOLDER+'velo_lidar.txt')
    os.remove(FOLDER+'zed_right.txt')
    os.remove(FOLDER+'meta.json')
    os.remove(FOLDER+'lidar_to_camera_frame.txt')


except:
    print("Files are already deleted")

Files are already deleted


In [16]:
#FOLDERS = ['city_1_0','city_1_1','city_1_3','city_2_0',
#           'junction_1_0','junction_1_1','junction_1_2',
#           'motorway_1_0','motorway_2_0','motorway_2_1',
#           'rural_1_1','rural_1_3','fog_6_0','fog_8_0',
#           'fog_8_1','fog_8_2','night_1_0','night_1_1',
#           'night_1_2','night_1_3','rain_2_0','rain_3_0',
#           'rain_4_0','rain_4_1','snow_1_0']
FOLDERS = [FOLDER[22:]]

## Create coco_annotation json file 

In [ ]:
import os
import json
import cv2
from tqdm import tqdm


for FOLDER in tqdm(FOLDERS):
    FOLDER = 'Radiate_Image_Dataset/'+ FOLDER+"/"
    img_folder = FOLDER + "fused_images/RGB/img/"
    annotation_folder = FOLDER + "coco_annotations"
    img_list = os.listdir(img_folder)
    annotation_list = os.listdir(annotation_folder)

    #Create the basic info of the annotation file
    dic = {
    "info":
        {"year":"2023",
        "version":"1",
        "description":"Radiate Dataset",
        "contributor":"Heriot-Watt University",
        "url":"https://pro.hw.ac.uk/radiate/",
        "date_created":"2020-03-15"},
    "licenses":
        [{"id":1,
        "url":"https://choosealicense.com/licenses/mit/",
        "name":"MIT"}],
    "categories":[
        {"id":0,
        "name":"car",
        "supercategory":"none"},
        {"id":1,
        "name":"van",
        "supercategory":"none"},
        {"id":2,
        "name":"truck",
        "supercategory":"none"},
        {"id":3,
        "name":"bus",
        "supercategory":"none"},
        {"id":4,
        "name":"motorbike",
        "supercategory":"none"},
        {"id":5,
        "name":"pedestrian",
        "supercategory":"none"},
        {"id":6,
        "name":"group_of_pedestrians",
        "supercategory":"none"},
        {"id":7,
        "name":"bicycle",
        "supercategory":"none"}],
    "images":[],
    "annotations":[]}


    #Add all the images
    for img in img_list:
        if img == '_annotations.coco.json':
            continue
        im = cv2.imread(img_folder+img)
        temp_dic = {
            "id":len(dic['images']),
            "license":1,
            "file_name":img,
            "height":im.shape[0],
            "width":im.shape[1],
            "date_captured":"2020-03-15"}  
        dic['images'].append(temp_dic)

    #Add the bounding boxes
    for i in range(len(annotation_list)-1):
        file_path = os.path.join(annotation_folder, annotation_list[i])
        if (annotation_list[i]== "only_with_annotations.txt"):
            continue
        with open(file_path,'r') as file:
            #search for image id
            for idx in range(len(dic['images'])):
                if annotation_list[i][:-4] == dic['images'][idx]["file_name"][:-4]:
                    index = idx

            bbox_list= file.read().split("\n")
            for j in range(len(bbox_list)-1):
                box = bbox_list[j]
                box = box.split(" ")
                box[1],box[2],box[3],box[4]=int(box[1]),int(box[2]),int(box[3]),int(box[4])
                #in case a bounding box is out of the boundaries of the image
                img_height = dic['images'][0]['height']
                img_width = dic['images'][0]['width']
                if box[1]<0:
                    w = abs(box[1]+box[3])
                    box[1] = 0
                    box[3] = w 
                if box[2]<0:
                    h = abs(box[2] +box[4])
                    box[2]=0
                    box[4] = h
                
                if box[1]>img_width:
                    box[1]=img_width
                if box[2]>img_height:
                    box[2]=img_height

                if box[1]+box[3]>img_width:
                    box[3]=img_width-box[1]
                if box[2]+box[4]>img_height:
                    box[4]=img_height-box[2]

                if box[0]=='None':
                    box[0]='7'
                
                temp_dic={
                    "id":len(dic['annotations']),
                    "image_id":dic['images'][index]['id'],
                    "category_id":box[0],
                    "bbox":[int(box[1]),int(box[2]),int(box[3]),int(box[4])],
                    "area":int(box[3])*int(box[4]),
                    "segmentation":[],
                    "iscrowd":0}
                if temp_dic['image_id']<0:
                    temp_dic['image_id']=0
                dic['annotations'].append(temp_dic)


    json_object = json.dumps(dic, indent=5)
    
    with open(FOLDER+"fused_images/RGB/_annotations.coco.json", "w") as outfile:
        outfile.write(json_object)

    with open(FOLDER+"fused_images/RGB_distance/_annotations.coco.json", "w") as outfile:
        outfile.write(json_object)

    

100%|██████████| 1/1 [00:00<00:00,  6.09it/s]


In [18]:
from PIL import Image, ImageDraw
import json
import random
import numpy as np
import torchvision.transforms as transforms 
from tqdm import tqdm
import os

for FOLDER in FOLDERS:
    FOLDER = 'Radiate_Image_Dataset/'+ FOLDER+"/"
    try:
        os.mkdir(FOLDER+'Annotated')
    except:
        print(FOLDER+'Annotated  already exists')


for FOLDER in tqdm(FOLDERS):
    FOLDER = 'Radiate_Image_Dataset/'+ FOLDER+"/"
    img_folder = FOLDER + "fused_images/RGB/img/"
    annotation_folder = FOLDER + "coco_annotations"
    img_list = os.listdir(img_folder)
    annotation_list = os.listdir(annotation_folder)

    json_path = FOLDER+"fused_images/RGB/_annotations.coco.json"
    with open(json_path, "r") as file:
        data_dic = json.load(file)
        

    #random_idx = random.randint(0, len(data_dic['images'])-1)

    for random_idx in range(len(data_dic['images'])):

        random_img = data_dic['images'][random_idx]

        img_path = img_folder +random_img['file_name']
        image = Image.open(img_path)

        image_array = np.asarray(image)
        image_array = image_array[:,:,:3]


        transform = transforms.ToPILImage()
        final_image = transform(image_array)

        draw = ImageDraw.Draw(final_image)

        random_img_id = random_img['id']

        for ann in data_dic['annotations']:
            if ann['image_id']==random_img_id:
                xmin,ymin,width,height = ann['bbox'][0], ann['bbox'][1], ann['bbox'][2],ann['bbox'][3]
                draw.rectangle([xmin, ymin, xmin + width, ymin + height], outline='green', width=2)

        final_image.save(f"{FOLDER}Annotated/Image #{random_idx}.jpg")

100%|██████████| 1/1 [00:00<00:00, 11.84it/s]


## Split train-test-val

In [19]:
import random
import json
from PIL import Image
import os 
import glob
from tqdm import tqdm

EXPERIMENTS = ['RGB/','RGB_distance/']


for FOLDER in tqdm(FOLDERS):
    FOLDER = 'Radiate_Image_Dataset/'+ FOLDER+"/"
    json_path = FOLDER+"fused_images/RGB/_annotations.coco.json"
    with open(json_path, "r") as file:
        data_dic = json.load(file)

    random.seed(42)
    random.shuffle(data_dic['images'])

    #Shuffle and Separate train, test, val
    train_percent = 0.70 
    valid_percent = 0.20 
    test_percent = 0.10

    total = len(data_dic['images'])
    train = round(train_percent*total)
    test = round(test_percent*total)
    valid = round(valid_percent*total)


    train_set = data_dic['images'][:train]
    test_set = data_dic['images'][train:train+test]
    valid_set = data_dic['images'][train+test:]

    train_dic = {
    "info":
        {"year":"2023",
        "version":"1",
        "description":"Radiate Dataset",
        "contributor":"Heriot-Watt University",
        "url":"https://pro.hw.ac.uk/radiate/",
        "date_created":"2020-03-15"},
    "licenses":
        [{"id":1,
        "url":"https://choosealicense.com/licenses/mit/",
        "name":"MIT"}],
    "categories":[
        {"id":0,
        "name":"car",
        "supercategory":"none"},
        {"id":1,
        "name":"van",
        "supercategory":"none"},
        {"id":2,
        "name":"truck",
        "supercategory":"none"},
        {"id":3,
        "name":"bus",
        "supercategory":"none"},
        {"id":4,
        "name":"motorbike",
        "supercategory":"none"},
        {"id":5,
        "name":"pedestrian",
        "supercategory":"none"},
        {"id":6,
        "name":"group_of_pedestrians",
        "supercategory":"none"},
        {"id":7,
        "name":"bicycle",
        "supercategory":"none"}],
    "images":[],
    "annotations":[]}

    test_dic = train_dic.copy()
    valid_dic = train_dic.copy()

    train_dic['images']=train_set
    test_dic['images']=test_set
    valid_dic['images']=valid_set

    temp_ann = []
    for img in train_set:
        for ann in data_dic['annotations']:
                if ann['image_id'] == img['id']:
                     temp_ann.append(ann)
    train_dic['annotations'] = temp_ann
    
    temp_ann = []
    for img in test_set:
        for ann in data_dic['annotations']:
                if ann['image_id'] == img['id']:
                    temp_ann.append(ann)
    test_dic['annotations'] = temp_ann

    temp_ann = []
    for img in valid_set:
        for ann in data_dic['annotations']:
                if ann['image_id'] == img['id']:
                    temp_ann.append(ann)
    valid_dic['annotations'] = temp_ann

    #Save annotations in folders  
    train_json = json.dumps(train_dic, indent=5)
    test_json = json.dumps(test_dic, indent=5)
    valid_json = json.dumps(valid_dic, indent=5) 

    for EXPERIMENT in EXPERIMENTS:
        try:
            os.mkdir(FOLDER+"fused_images/"+EXPERIMENT+"train")
            os.mkdir(FOLDER+"fused_images/"+EXPERIMENT+"test")
            os.mkdir(FOLDER+"fused_images/"+EXPERIMENT+"valid")
        except:
            files = glob.glob(FOLDER+"fused_images/"+EXPERIMENT+"train/*")
            for f in files:
                 os.remove(f)
            files = glob.glob(FOLDER+"fused_images/"+EXPERIMENT+"test/*")
            for f in files:
                 os.remove(f)
            files = glob.glob(FOLDER+"fused_images/"+EXPERIMENT+"valid/*")
            for f in files:
                 os.remove(f)

        #Copy images to folders and get annotations
        for img in train_set:
            img_path = FOLDER+"fused_images/"+EXPERIMENT+"img/"+img['file_name']
            with Image.open(img_path) as image:
                    image.save(FOLDER+"fused_images/"+EXPERIMENT+'train/'+img['file_name'])

        for img in test_set:
            img_path = FOLDER+"fused_images/"+EXPERIMENT+"img/"+img['file_name']
            with Image.open(img_path) as image:
                    image.save(FOLDER+"fused_images/"+EXPERIMENT+'test/'+img['file_name'])

        for img in valid_set:
            img_path = FOLDER+"fused_images/"+EXPERIMENT+"img/"+img['file_name']
            with Image.open(img_path) as image:
                    image.save(FOLDER+"fused_images/"+EXPERIMENT+'valid/'+img['file_name'])


        with open(FOLDER+"fused_images/"+EXPERIMENT+"train/_annotations.coco.json", "w") as file:
            file.write(train_json)

        with open(FOLDER+"fused_images/"+EXPERIMENT+"test/_annotations.coco.json", "w") as file:
            file.write(test_json)

        with open(FOLDER+"fused_images/"+EXPERIMENT+"valid/_annotations.coco.json", "w") as file:
            file.write(valid_json)
    print(f'Total:{total}, Train:{train}, Test:{test}, Validation:{valid}')

100%|██████████| 1/1 [00:01<00:00,  1.13s/it]

Total:14, Train:10, Test:1, Validation:3
